In [ ]:
from chxanalys.chx_packages import *
%matplotlib notebook
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams.update({ 'image.origin': 'lower'   })
plt.rcParams.update({ 'image.interpolation': 'none'   })
import pickle as cpk
from chxanalys.chx_xpcs_xsvs_jupyter_V1 import *

In [ ]:
%matplotlib inline

In [ ]:
scat_geometry ='saxs'
qphi_analysis = False
run_t_ROI_Inten = True
use_sqnorm = True
use_imgsum_norm = True
run_one_time = True
bad_frame_list = None
good_start = 0
fit_g2_func = 'stretched'

run_two_time =   True    #run  two-time
run_four_time = False #True #False   #run  four-time
run_xsvs=  False #False         #run visibility analysis
att_pdf_report = True    #attach the pdf report to CHX olog
qth_interest = 1 #the intested single qth             
use_sqnorm = True    #if True, use sq to normalize intensity
use_imgsum_norm= True  #if True use imgsum to normalize intensity for one-time calculatoin
pdf_version='_%s'%get_today_date()     #for pdf report name
run_dose =  False #True  #run dose_depend analysis

if scat_geometry == 'gi_saxs':run_xsvs= False;use_sqnorm=False
if scat_geometry == 'gi_waxs':use_sqnorm = False;
if scat_geometry != 'saxs':qphi_analysis = False;scat_geometry_ = scat_geometry  
else:scat_geometry_ = ['','ang_'][qphi_analysis]+ scat_geometry


In [ ]:
#Sample info

T= True
F = False


#chx_THL230_15s_11_W0028_H11-171113-172644-1.csv
#--> count : 1 ['8452c7'] (scan num: 9195) (Measurement: Timpix3 1s series file chx_THL230_11s_10: exp_1s Silica 250 nm diam, stock solution in water )
    
sam = dict(  
run11 =  'chx_THL230_15s_11_W0028_H11-171113-172644-1.csv',
run16 = 'chx_THL230_15s_16_W0028_H11-171114-041148-1.csv', 
run20 = 'chx_THL230_3600s_20_W0028_H11-171114-063246.csv',    
    
mask  =  'W0028_H11_trimdacs.txt'
    
)    

#save_dict_csv(sam, out_dir0 + 'sample_info.csv')

data_dir = '/XF11ID/analysis/2017_3/Timepix/Data/'
out_dir0 = '/XF11ID/analysis/2017_3/Timepix/Results/'

out_dir0 = '/home/yuzhang/Analysis/Timepix/2017_3/Results/'




In [ ]:
cd /XF11ID/analysis/2017_3/Timepix/Data/

In [ ]:
ls

In [ ]:
#sam

In [ ]:
from chxanalys.chx_packages import *
%matplotlib notebook
plt.rcParams.update({'figure.max_open_warning': 0})
#%reset -f  #for clean up things in the memory
import pandas as pds

## Provide sample filename here

In [ ]:
run = '11'
thresh = 230
uid = 'run%s'%run
fp = sam[ uid ]
print( uid, fp  ) 

out_dir = os.path.join(out_dir0, '%s/'%uid)
os.makedirs(out_dir, exist_ok=True)
print('Results from this analysis will be stashed in the directory %s' %out_dir)
uidstr = 'uid=%s'%uid

# Get Data

In [ ]:
%run /XF11ID/analysis/Analysis_Pipelines/Develop/chxanalys/chxanalys/xpcs_timepixel.py

In [ ]:
time_unit = 25/4096/1e9#  * this time_unit to make scale in s
print(time_unit)


In [ ]:
x,y,t0 = get_timepixel_data( data_dir, fp,time_unit=1 )
t = t0 * time_unit
#print( 'The max time is %s ms' %( t.max()*6.1/1e9) )

In [ ]:
print( 'The time duration is %s s.' %t.max()  )

In [ ]:
#plot1D(t)

In [ ]:
#Due to shutter open chose data from 100 to -100

In [ ]:
if False:
    x = x[100:-100]
    y = y[100:-100]
    t = t[100:-100]

# Get Time~Intensity Plot

In [ ]:
tbin_step = 0.001 * 1  #in unit of s
times, imgsum = get_his_taus( t, bin_step = tbin_step )
#times in ms
print(times.shape) 
timeperframe =  tbin_step 
print ( timeperframe)

In [ ]:
plot1D( x = times, y= imgsum , m='o', c='b', ls = '--',save=True, path= out_dir,
     title='%s_imgsum_time'%uid, xlabel = r"$time $ $(s)$", ylabel = 'Photon Counts of Image Sum' ) 

In [ ]:
%run /XF11ID/analysis/Analysis_Pipelines/Develop/chxanalys/chxanalys/chx_generic_functions.py

In [ ]:
thres = 1.2e4
w= np.where( imgsum >thres )
t1,t2= times[ w[0][0] ], times[ w[0][-1] ]
t1_ind, t2_ind =  find_index(t,t1), find_index(t,t2)
print( t1,t2, t1_ind, t2_ind  )

# Get interested time range

In [ ]:
#t1_ind, t2_ind = 0, 1000

In [ ]:
xeff = x[t1_ind:t2_ind] 
yeff = y[t1_ind:t2_ind] 
poseff = xeff * 256 + yeff
t0eff =t0[t1_ind:t2_ind] - t0[t1_ind]
teff = t0eff * time_unit

In [ ]:
#tbin_step = 0.01 * 1  #in unit of s
times, imgsum = get_his_taus( teff, bin_step = tbin_step )
#times in ms
print(times.shape) 
timeperframe =  tbin_step 
print ( timeperframe)

plot1D( x = times, y= imgsum , m='o', c='b', ls = '--',save=True, path= out_dir,
     title='%s_imgsum_time'%uid, xlabel = r"$time $ $(s)$", ylabel = 'Photon Counts of Image Sum' ) 

# Get/Create Mask

In [ ]:
m = np.loadtxt( data_dir + sam['mask'])

In [ ]:
#mask = np.zeros( [256, 256] )
mask = 1 - m[:,3].reshape( [256, 256 ])

In [ ]:
edg = 2
mask[:edg,:] = 0
mask[-edg:,:] = 0
mask[:,:edg] = 0
mask[:,-edg:] = 0

In [ ]:
show_img( mask )

In [ ]:
%run /XF11ID/analysis/Analysis_Pipelines/Develop/chxanalys/chxanalys/xpcs_timepixel.py

# Get Averaged Image

In [ ]:
avg_img = get_timepixel_avg_image( xeff,yeff,teff,  delta_time = None ) * mask

In [ ]:
show_img( avg_img,  alpha=0.8, vmin=0.0001, vmax= 5e9, logs=True, aspect=1,  
          
         image_name= '%s_img_avg'%uid,  save=True, path= out_dir,  cmap = cmap_albula )

In [ ]:
center = [ 256, 280 ] #center0 is the image x, center1 is the image y  #actually this is good!

In [ ]:
rmk = create_ring_mask( [256, 256],  282, 292, center= center ) * mask
r1d = avg_img[np.array(rmk, dtype=bool) ].ravel()
plot1D(  r1d, c='b', m='o'  )

In [ ]:
#avg_img = get_timepixel_avg_image( xeff,yeff,teff,  delta_time = None )  

In [ ]:
show_img( avg_img, label_array = None, alpha=0.4, vmin=5, vmax= 5e4, logs=True, aspect=1, center= center[::-1],           
         image_name= '%s_img_avg'%uid,  save=True, path= out_dir,  cmap = cmap_albula )

# Do a circular average

In [ ]:
center

In [ ]:
out_dir

In [ ]:
setup_pargs = {'Ldet': 16035 + 0,  'center': center,
               'dpix': 0.046,  #what's pixel size, should be 45 um???
               'exposuretime':  0.001, 'lambda_': 1.28481,
            'path': out_dir,  
               'uid': 'uid=Run11'}

In [ ]:
#avg_img_ = FD.rdframe(0) * mask

In [ ]:
#avg_img_ =  get_avg_imgc(FD,sampling=1)

In [ ]:
qp_saxs, iq_saxs, q_saxs = get_circular_average( avg_img, mask, pargs=setup_pargs  )
plot_circular_average( qp_saxs, iq_saxs, q_saxs,  pargs=setup_pargs,  
                  xlim=[q_saxs.min(), q_saxs.max()*1.0], ylim = [iq_saxs.min(), iq_saxs.max()] )

In [ ]:
iqdir = '/XF11ID/analysis/2017_3/yuzhang/Results/cb9605/'
fuid = 'cb9605fa-b9ea-41fb-afb1-3940ecc0fa84'
extract_dict = extract_xpcs_results_from_h5( filename = 'uid=%s_Res.h5'%fuid, import_dir = iqdir )

In [ ]:
extract_dict.keys()

In [ ]:
fig, ax = plt.subplots()
plot1D( x= extract_dict['q_saxs'], y= extract_dict[ 'iq_saxs' ], ax =ax, legend='4M', m='s',c='k' )
plot1D( x=q_saxs, y= iq_saxs * 100, ax =ax, legend='Timepix', m='o',c='b', logy=True, xlim=[0., 0.01], ylim=[0.01,100] )
ax.vlines( 0.00316, 0.,10 )


# Create ring-shaped ROI

In [ ]:

if scat_geometry =='saxs':    
    uniformq =  True  #True # False    
    ## Define ROI
    #* Create ring mask defined by  inner_radius, outer_radius, width, num_rings (all in pixel unit)
    #* Create ring mask defined by  edges (all in pixel unit)    
    ### Define a non-uniform distributed rings by giving edges
    if not uniformq:
        
        qcenters = [  0.0023,  0.00365, 0.0050, ]#0.00621, 0.00754, 0.00880  ] #in A-1        
        #width = 0.0001  # in A-1         
        #width =    [0.0001,      0.00012,  0.00014,  0.00016, 0.00018,  0.0002,  0.00022 ]
        width =    np.array( [0.0001,      0.00012,  0.00014,  0.00016, 0.00018,  0.0002,  0.00022 ] ) * 3.5
        
        edges = get_non_uniform_edges(  qcenters, width, number_rings =1 )    
        inner_radius= None
        outer_radius = None
        width = None
        num_rings = None        
    # Define a uniform distributed rings by giving inner_radius, outer_radius, width, num_rings (all in pixel unit)
    if uniformq:            
        inner_radius= 0.0026 #0.006  #16
        outer_radius = 0.0050 #0.05  #112    
        num_rings = 12 #18
        gap_ring_number = 0.1
        width =    ( outer_radius - inner_radius)/(num_rings + gap_ring_number)
        edges = None





In [ ]:
if scat_geometry =='saxs':
    roi_mask, qr, qr_edge = get_ring_mask(  mask, inner_radius=inner_radius, 
            outer_radius = outer_radius , width = width, num_rings = num_rings, edges=edges,
                          unit='A',       pargs=setup_pargs   )
    qind, pixelist = roi.extract_label_indices(  roi_mask  ) 
    qr = np.round( qr, 5)
    print(len(qr))
    
    #show_ROI_on_image( avg_img, roi_mask, center, label_on = False, rwidth = 140, alpha=.9,  
    #                 save=True, path=data_dir, uid=uidstr, vmin= np.min(avg_img), vmax= np.max(avg_img),
    #                 aspect=1) 
    qval_dict = get_qval_dict( np.round(qr, 5)  )

In [ ]:
#show_img(roi_mask * mask)

In [ ]:
show_img( avg_img, label_array = roi_mask, alpha=0.4, vmin=5, vmax= 5e5, logs=True, aspect=1, center= center[::-1],           
         image_name= '%s_img_avg'%uid,  save=True, path= out_dir,  cmap = cmap_albula )

In [ ]:
# plot_qIq_with_ROI??

In [ ]:
#show_img( avg_img, label_array = roi_mask, alpha=0.4, vmin=5, vmax= 5e5, logs=True, aspect=1, center= center[::-1],           
#         image_name= '%s_img_avg'%uid,  save=True, path= out_dir,  cmap = cmap_albula )

In [ ]:
if scat_geometry =='saxs':
    plot_qIq_with_ROI( q_saxs, iq_saxs, qr, logs=True, uid=uidstr, xlim=[0.002,0.006],
                  ylim = [iq_saxs.min(), iq_saxs.max()*2],  save=True, path= out_dir)

# Compress Data

In [ ]:
xeff,yeff,t0eff

In [ ]:
%run /XF11ID/analysis/Analysis_Pipelines/Develop/chxanalys/chxanalys/xpcs_timepixel.py

In [ ]:
tbins = 0.01 #10 ms
tbins = 0.001 #1 ms
tbins = 0.0001 #0.1 ms
tbins = 0.00001  #0.01 ms

tbins = 0.000001  #0.001 ms, 1 microsecond

tbins = 1e-6 *5
binstep = int(tbins/time_unit)
tinbs = binstep * time_unit
    
print(binstep, tinbs)




data_dir



tot =  pds.read_csv( data_dir + fp )[' #ToT']
toteff = np.array(tot)[t1_ind:t2_ind]

#plot1D( np.bincount(toteff) )

#w = np.where( (toteff >=150) & (toteff <=500) )[0]

#good_time_ind = np.where(diff>500*1e-9)[0]
#good_time_ind = np.where( tot*time_unit > 500*1e-9)[0]
#good_time_ind = np.where(diff>0)[0]

good_time_ind = np.where( (toteff >= 300) & (toteff <= 400) )[0]

good_time_ind

#len( np.where( diff <=500 )[0] )/len(t0eff)

len(good_time_ind)/len(t0eff)



In [ ]:
run = '11'
run = '16'
run = '20'
thresh = 230
uid = 'run%s'%run
fp = sam[ uid ]
print( uid, fp  ) 

out_dir_ = os.path.join(out_dir0, '%s/'%uid)
os.makedirs(out_dir_, exist_ok=True)
print('Results from this analysis will be stashed in the directory %s' %out_dir_)
uidstr = 'uid=%s'%uid

# Get Data

In [ ]:
time_unit = 25/4096/1e9#  * this time_unit to make scale in s
print(time_unit)
x,y,tt0 = get_timepixel_data( data_dir, fp,time_unit=1 )
t = tt0 * time_unit
#print( 'The max time is %s ms' %( t.max()*6.1/1e9) )

In [ ]:
tbin_step = 0.001 * 1  #in unit of s
times0, imgsum0 = get_his_taus( t, bin_step = tbin_step )
#times in ms
print(times0.shape) 
timeperframe =  tbin_step 
print ( timeperframe)

# Do loop here for one long count with different time ROI

In [ ]:
thres = 2000
wimgsum= np.where( imgsum0 > thres )[0]
wtimes = times0[wimgsum]
ww = np.where( np.diff( wtimes ) >  1)[0]
wt = np.zeros( [   len(ww) +1, 2]  )
wt[0] = [times0[798], times0[wimgsum][ww][0]]
wt[1:-1,0] =  times0[wimgsum][ww+1][:-1]
wt[1:-1,1] =    times0[wimgsum][ww][1:]
wt[-1] =  [ times0[wimgsum][ww+1][-1], times0[wimgsum][ww+1][-1] +3]
#print( wt )

#### do loop here

In [ ]:
out_dir0

In [ ]:
out_dir_

In [ ]:
%run /home/yuzhang/chxanalys_link/chxanalys/Create_Report.py

In [ ]:
for posi in  np.arange(15,25):  
    print(posi)

In [ ]:
#for posi in  np.arange(60):
for posi in  np.arange(25,60):    
    
    taus=None;g2=None;tausb=None;g2b=None;
    g12b=None;taus4=None;g4=None;times_xsv=None;
    contrast_factorL=None; lag_steps = None;
    data_pixel=None;
    
    
    print('*'*50)
    print('Running for the %s position'%posi)
    #for posi in [0]:  
    tbins = 0.00001  #10 us
    binstep = int(tbins/time_unit)
    tinbs = binstep * time_unit
    print(binstep, tinbs)    
    
    uid='run20_pos%s'%(posi+1)
    print(uid)
    out_dir = os.path.join(out_dir_, '%s/'%uid)
    os.makedirs(out_dir, exist_ok=True)
    print('Results from this analysis will be stashed in the directory %s' %out_dir)
    uidstr = 'uid=%s'%uid

    tw1, tw2 = wt[posi] 
    #tw1, tw2 = 0.79, 0.80 #for test purpose
    
    t1_ind, t2_ind =  np.argmin(np.abs(t-tw1)),np.argmin(np.abs(t-tw2))
    #print(t1_ind, t2_ind)
    print(tw1, tw2)
    print(t[t1_ind], t[t2_ind])


    xeff = x[t1_ind:t2_ind] 
    yeff = y[t1_ind:t2_ind] 
    poseff = xeff * 256 + yeff
    t0eff =tt0[t1_ind:t2_ind] - tt0[t1_ind]
    teff = t0eff * time_unit    
       
    md = { 'detector_distance': 16035 + 0,
          'beam_center_x': 256,
          'beam_center_y': 280,
           'x_pixel_size': 0.045,
          'y_pixel_size': 0.0045,
          'incident_wavelength': 1.28481,
          'sx': 256,
          'sy':256,
         'frame_time': tbins, 
          'count_time': tbins,
           'uid': 'uid=run20_pos%s'%(1+posi)
         }

    setup_pargs = {'Ldet': 16035 + 0,  'center': center,
               'dpix': 0.046,  #what's pixel size, should be 45 um???
               'exposuretime':  0.001, 'lambda_': 1.28481,
            'path': out_dir,  
               'uid': 'uid=run20_pos%s'%(1+posi) }
    
    filename = out_dir0 + uid + '_tbin=%s.cmp'%tbins
    if False:#consider dead time issue
        avg_img, imgsum, N = compress_timepix_data( poseff[good_time_ind],
                                               t0eff[good_time_ind],
                                               binstep, filename = filename, 
                                               md = md, force_compress= True  )
    else:        
        avg_img, imgsum, N = compress_timepix_data( poseff,
                                                    t0eff,
                                               binstep, filename = filename, 
                                               md = md, force_compress= True  )
        
    print(N)
    FD = Multifile(filename, 0, N  )
    uidstr_ = uidstr + 'tbins=%.3f_ms'%( tbins*1000 ) 
    
    good_start = 0
    bad_frame_list =[]
    plot1D( y = imgsum[ np.array( [i for i in np.arange(good_start, len(imgsum)) if i not in bad_frame_list])],
       title =uidstr + '_imgsum', xlabel='Frame', ylabel='Total_Intensity', legend='imgsum'   )
    Nimg = N
    show_img( avg_img,   vmin=None, vmax=None, logs=True, aspect=1, #save_format='tif',
         image_name= uidstr + '_img_avg',  save=True, path=out_dir,  cmap = cmap_albula, center=center[::-1] )
    
    bad_frame_list =  get_bad_frame_list( imgsum, fit='both',  plot=True,polyfit_order = 30,                                      
                        scale= 3.5,  good_start = good_start, good_end=Nimg, uid= uidstr, path=out_dir)
    print( 'The bad frame list length is: %s'%len(bad_frame_list) )        
    
    imgsum_y = imgsum[ np.array( [i for i in np.arange( len(imgsum)) if i not in bad_frame_list])]
    imgsum_x = np.arange( len( imgsum_y))
    save_lists(  [imgsum_x, imgsum_y], label=['Frame', 'Total_Intensity'],
           filename=uidstr + '_img_sum_t', path= data_dir  )
    
    plot1D( y = imgsum_y, title = uidstr + '_img_sum_t', xlabel='Frame', c='b',
           ylabel='Total_Intensity', legend='imgsum', save=True, path=out_dir)
    
    if scat_geometry =='saxs':
        ## Get circular average| * Do plot and save q~iq
        #hmask = create_hot_pixel_mask( avg_img, threshold = 1e2, center=center, center_radius= 100)
        #mask = mask * hmask
        qp_saxs, iq_saxs, q_saxs = get_circular_average( avg_img, mask, pargs= setup_pargs  )
        plot_circular_average( qp_saxs, iq_saxs, q_saxs,  pargs=setup_pargs, 
                          xlim=[q_saxs.min(), q_saxs.max()*1.0], ylim = [iq_saxs.min(), iq_saxs.max()] )
    #mask =np.array( mask * hmask, dtype=bool)  

    show_img( avg_img, label_array = roi_mask, alpha=0.4, vmin=None, vmax= None, logs=True, aspect=1, center= center[::-1],           
         image_name= '%s_ROI_on_Image'%uidstr,  save=True, path= out_dir,  cmap = cmap_albula )
    if scat_geometry =='saxs':
        plot_qIq_with_ROI( q_saxs, iq_saxs, qr, logs=True, uid=uidstr, xlim=[0.002,0.006],
                  ylim = [iq_saxs.min(), iq_saxs.max()*2],  save=True, path= out_dir)  
    if scat_geometry =='saxs':
        Nimg = FD.end - FD.beg 
        time_edge = create_time_slice( Nimg, slice_num= 4, slice_width= 2, edges = None )
        time_edge =  np.array( time_edge ) #+  good_start
        #print( time_edge )    
        qpt, iqst, qt = get_t_iqc( FD, time_edge, mask, pargs=setup_pargs, nx=1500, show_progress= False )
        plot_t_iqc( qt, iqst, time_edge, pargs=setup_pargs, xlim=[qt.min(), qt.max()],
               ylim = [iqst.min(), iqst.max()], save=True )        

    qind, pixelist = roi.extract_label_indices(roi_mask)
    noqs = len(np.unique(qind))
    nopr = np.bincount(qind, minlength=(noqs+1))[1:]
    nopr
    ring_avg = None    
    if run_t_ROI_Inten:
        times_roi, mean_int_sets = cal_each_ring_mean_intensityc(FD, roi_mask, timeperframe = None, multi_cor=True  ) 
        plot_each_ring_mean_intensityc( times_roi, mean_int_sets,  uid = uidstr, save=True, path= out_dir )
        roi_avg = np.average( mean_int_sets, axis=0)
    
    # Check one ROI intensity
    roi_inten = check_ROI_intensity( avg_img, roi_mask, ring_number= 7, uid =uidstr_ ) #roi starting from 1
    # Do one time
    if use_sqnorm:norm = get_pixelist_interp_iq( qp_saxs, iq_saxs, roi_mask, center)
    else:norm=None 
    if use_imgsum_norm:imgsum_ = imgsum
    else:imgsum_ = None    
    import time
    if run_one_time: 
        t0 = time.time()    
        g2, lag_steps  = cal_g2p( FD,  roi_mask, bad_frame_list,good_start, num_buf = 8, num_lev= None,
                                imgsum= imgsum_, norm=norm )
        run_time(t0)

    lag_steps = lag_steps[:g2.shape[0]]
    uid_ = uidstr + '_fra_%s_%s_tbins=%.3f_ms'%(FD.beg, FD.end,  tbins*1000 )
    timeperframe  = tbins
    print(uid_)

    if run_one_time:

        taus = lag_steps * timeperframe    
        try:
            g2_pds = save_g2_general( g2, taus=taus,qr= np.array( list( qval_dict.values() ) )[:,0],
                                                qz = np.array( list( qval_dict.values() ) )[:,1],
                                 uid=uid_+'_g2.csv', path= out_dir, return_res=True )
        except:
            g2_pds = save_g2_general( g2, taus=taus,qr= np.array( list( qval_dict.values() ) )[:,0],                                             
                                 uid=uid_+'_g2.csv', path= out_dir, return_res=True )

    if run_one_time:
        g2_fit_result, taus_fit, g2_fit = get_g2_fit_general( g2,  taus, 
                    function = fit_g2_func,  vlim=[0.95, 1.05], fit_range= None,  
                fit_variables={'baseline':True, 'beta': True, 'alpha':False,'relaxation_rate':True},                                  
                guess_values={'baseline':1.0,'beta': 0.1,'alpha':1.0,'relaxation_rate':0.0100,},
                guess_limits = dict( baseline =[1, 1.8], alpha=[0, 2],
                            beta = [0, 1], relaxation_rate= [0.00001, 5000]) ) 
        g2_fit_paras = save_g2_fit_para_tocsv(g2_fit_result,  filename= uid_  +'_g2_fit_paras.csv', path= out_dir )

     

    if run_one_time:
        plot_g2_general( g2_dict={1:g2, 2:g2_fit}, taus_dict={1:taus, 2:taus_fit}, vlim=[0.95, 1.05],
                    qval_dict = qval_dict, fit_res= g2_fit_result,  geometry= scat_geometry_,filename= uid_+'_g2', 
            path= out_dir, function= fit_g2_func,  ylabel='g2', append_name=  '_fit')
  

    if run_one_time:
        if False:
            fs, fe = 0, 9
            fs,fe=0, 12
            qval_dict_ = {k:qval_dict[k] for k in list(qval_dict.keys())[fs:fe]  }
            D0, qrate_fit_res = get_q_rate_fit_general(  qval_dict_, g2_fit_paras['relaxation_rate'][fs:fe], 
                                                       geometry=  scat_geometry_ )
            plot_q_rate_fit_general( qval_dict_, g2_fit_paras['relaxation_rate'][fs:fe],  qrate_fit_res, 
                                    geometry= scat_geometry_,uid=uid_  , path= out_dir )
        else:
            D0, qrate_fit_res = get_q_rate_fit_general(  qval_dict, g2_fit_paras['relaxation_rate'],
                                        fit_range=[0, 26],   geometry= scat_geometry_ )    
            plot_q_rate_fit_general( qval_dict, g2_fit_paras['relaxation_rate'],  qrate_fit_res,   
                                geometry=  scat_geometry_,uid=uid_  ,
                                    show_fit= True, path= out_dir, plot_all_range=False)

    # Calculation of g2
    if False:
        get_diffusion_coefficient(   8.9*1e-4, 125*1e-9, T=298)/1e8 

        num_bufs=8
        noframes =  1e6
        num_levels = int(np.log( noframes/(num_bufs-1))/np.log(2) +1) +1
        tot_channels, lag_steps2, dict_lag = multi_tau_lags(num_levels, num_bufs)
        max_taus= lag_steps2.max()
        taus_ = lag_steps2 * 1e-7
        g2_q1 = cal_particle_g2( radius=125*1e-9, viscosity=8.9*1e-4, qr=qr,  taus=taus_, beta=0.214, T=298)

        fig, ax = plt.subplots()
        for i, q1 in enumerate(qr):
            plot1D( x=taus_, y= g2_q1[i],   ls='-', m='', logx=True, ax =ax, c=colors[i],  legend=qr[i])
            plot1D( x=taus[1:], y= g2[1:,i], m = markers[i], ls='', logx=True, xlim=[0.000008, 0.01], ylim=[0.98,1.22],
                   ax =ax, c=colors[i],   legend= '')
        fig.savefig(out_dir + uid_ + 'g2_with_cal.png')

    # Two time
    
    define_good_series = False
    define_good_series = True
    if define_good_series:
        good_start = 5
        FD2 = Multifile(filename, beg = good_start, end = min(20000,Nimg) )
        uid_ = uidstr + '_fra_%s_%s_tbins=%.3f_ms'%(FD2.beg, FD2.end,  tbins*1000 )
        print( uid_ )
    else:
        FD2 = FD 

    data_pixel = None
    if run_two_time:    
        data_pixel =   Get_Pixel_Arrayc( FD2, pixelist,  norm= norm ).get_data()

    import time
    t0=time.time()
    g12b=None
    if run_two_time:     
        g12b = auto_two_Arrayc(  data_pixel,  roi_mask, index = None   )
        if run_dose:
            np.save( out_dir + 'uid=%s_g12b'%uid, g12b)
    run_time( t0 )

    if run_two_time:
        show_C12(g12b, q_ind= 6, N1= FD2.beg,logs=False, N2=min( FD2.end,100000), vmin= 1.01, vmax=1.4, 
             timeperframe=timeperframe,save=True, path= out_dir, uid = uid_ )

    multi_tau_steps = True
    if run_two_time:
        if lag_steps is None:
            num_bufs=8
            noframes = FD2.end - FD2.beg
            num_levels = int(np.log( noframes/(num_bufs-1))/np.log(2) +1) +1
            tot_channels, lag_steps, dict_lag = multi_tau_lags(num_levels, num_bufs)
            max_taus= lag_steps.max()

        #max_taus= lag_steps.max()  
        max_taus = N #Nimg    
        t0=time.time()
        #tausb = np.arange( g2b.shape[0])[:max_taus] *timeperframe
        if multi_tau_steps:
            lag_steps_ = lag_steps[   lag_steps <= g12b.shape[0] ]
            g2b = get_one_time_from_two_time(g12b)[lag_steps_]
            tausb = lag_steps_ *timeperframe
        else:
            tausb = (np.arange( g12b.shape[0]) *timeperframe)[:-200]
            g2b = (get_one_time_from_two_time(g12b))[:-200]
        run_time(t0)

        g2b_pds = save_g2_general( g2b, taus=tausb, qr= np.array( list( qval_dict.values() ) )[:,0],
                                  qz=None, uid=uid_ +'_g2b.csv', path= out_dir, return_res=True )

    if run_two_time:    
        g2b_fit_result, tausb_fit, g2b_fit = get_g2_fit_general( g2b,  tausb, 
                    function = fit_g2_func,  vlim=[0.95, 1.05], fit_range= None,  
                fit_variables={'baseline':False, 'beta': True, 'alpha':False,'relaxation_rate':True},                                  
                guess_values={'baseline':1.0,'beta': 0.15,'alpha':1.0,'relaxation_rate':1,},
                guess_limits = dict( baseline =[1, 1.8], alpha=[0, 2],
                            beta = [0, 1], relaxation_rate= [0.000001, 5000]) ) 
        g2b_fit_paras = save_g2_fit_para_tocsv(g2b_fit_result,  filename= uid_  +'_g2b_fit_paras.csv', path= out_dir )

    if run_two_time:
        plot_g2_general( g2_dict={1:g2b, 2:g2b_fit}, taus_dict={1:tausb, 2:tausb_fit}, vlim=[0.95, 1.05],
                    qval_dict=qval_dict, fit_res= g2b_fit_result,  geometry=scat_geometry_,filename=uid_+'_g2', 
                        path= out_dir, function= fit_g2_func,  ylabel='g2', append_name=  '_b_fit')

    if run_two_time:    
        if False:
            fs, fe = 0,9
            fs, fe = 0,12
            qval_dict_ = {k:qval_dict[k] for k in list(qval_dict.keys())[fs:fe]  }
            D0b, qrate_fit_resb = get_q_rate_fit_general(  qval_dict_, g2b_fit_paras['relaxation_rate'][fs:fe], geometry= scat_geometry_ )
            plot_q_rate_fit_general( qval_dict_, g2b_fit_paras['relaxation_rate'][fs:fe],  qrate_fit_resb, 
                                geometry= scat_geometry_,uid=uid_ +'_two_time' , path= out_dir )
        else:

            D0b, qrate_fit_resb = get_q_rate_fit_general(  qval_dict, g2b_fit_paras['relaxation_rate'],
                                            fit_range=[1, 10],  geometry= scat_geometry_ )
            plot_q_rate_fit_general( qval_dict, g2b_fit_paras['relaxation_rate'],  qrate_fit_resb,   
                                geometry= scat_geometry_,uid=uid_ +'_two_time', show_fit=False,path= out_dir, plot_all_range= True )

    if run_two_time and run_one_time:
        plot_g2_general( g2_dict={1:g2, 2:g2b}, taus_dict={1:taus, 2:tausb},vlim=[0.99, 1.007],
                    qval_dict=qval_dict, g2_labels=['from_one_time', 'from_two_time'],
                geometry=scat_geometry_,filename=uid_+'_g2_two_g2', path= out_dir, ylabel='g2', )


        
    #pdf_out_dir = os.path.join('/XF11ID/analysis/', CYCLE, username, 'Results/')
    pdf_out_dir = out_dir_
    pdf_filename = "XPCS_Analysis_Report_for_uid=%s%s_tbin=%s.pdf"%(uid,pdf_version, tbins)

    if run_xsvs:
        pdf_filename = "XPCS_XSVS_Analysis_Report_for_uid=%s%s.pdf"%(uid,pdf_version)

    print(pdf_filename)    
    
    username = 'yuzhang'
    run_fit_form = False
    run_invariant_analysis = False
    md['beg'] = 0
    #Nimg = N
    md['end'] = Nimg
    md['number of images'] = Nimg
    md['uid_g2'] =   'run%s_pos%s_fra_%s_%s_tbins=%.3f_ms'%(run,posi+1,0,Nimg, tbins*1000)     
    md['uid_c12'] =  'run%s_pos%s_fra_%s_%s_tbins=%.3f_ms'%(run,posi+1,FD2.beg, FD2.end, tbins*1000)
    md['sample'] = 'SiO2_Janus_Dia250_inFuel_2%'
    md['roi_mask_file'] =  out_dir + 'ring_roi_2017_nov24.pkl'

    make_pdf_report( out_dir, uid, pdf_out_dir, pdf_filename, username, 
                    run_fit_form,run_one_time, run_two_time, run_four_time, run_xsvs, run_dose,
                report_type= scat_geometry, report_invariant= run_invariant_analysis,
               md = md )    
    
    print('*'*50)    
    print('Finish the running of %s position.'%posi)
    plt.close('all')
    xeff = 0
    yeff = 0
    poseff = 0
    t0eff = 0
    teff = 0
    
    

In [ ]:
plt.close('all')

In [ ]:
%run /home/yuzhang/chxanalys_link/chxanalys/Create_Report.py

In [ ]:
    username = 'yuzhang'
    run_fit_form = False
    run_invariant_analysis = False
    md['beg'] = 0
    Nimg = N
    md['end'] = Nimg
    md['number of images'] = Nimg
    md['uid_g2'] =  'run%s_pos%s_fra_%s_%s_tbins=%.3f_ms'%(run,posi+1,0,Nimg, tbins*1000)
    md['uid_c12'] =  'run%s_pos%s_fra_%s_%s_tbins=%.3f_ms'%(run,posi+1,FD2.beg, FD2.end, tbins*1000)
    md['sample'] = 'SiO2_Janus_Dia250_inFuel_2%'
    md['roi_mask_file'] =  out_dir + 'ring_roi_2017_nov24.pkl'

In [ ]:
pdf_out_dir = out_dir_

In [ ]:
    make_pdf_report( out_dir, uid, pdf_out_dir, pdf_filename, username, 
                    run_fit_form,run_one_time, run_two_time, run_four_time, run_xsvs, run_dose,
                report_type= scat_geometry, report_invariant= run_invariant_analysis,
               md = md ) 

In [ ]:
%run /home/yuzhang/chxanalys_link/chxanalys/Create_Report.py

# Do loop here for one uid with different bin time

In [ ]:
run_dose=False
run_two_time = False

In [ ]:
out_dir0

In [ ]:
for tbins in 0.001 * np.array( [10,1,   0.1,  0.01,  0.0075,    0.005,  0.0025, 0.001 ]):
                          #10ms, 1ms,  100 us, 10 us, 7.5 us, 5us,   2.5us, 1us

#for tbins in 0.001 * np.array( [ 0.0075  ]):        

    binstep = int(tbins/time_unit)
    tinbs = binstep * time_unit    
    print(binstep, tinbs)
       
    md = { 'detector_distance': 16035 + 0,
          'beam_center_x': 256,
          'beam_center_y': 280,
           'x_pixel_size': 0.045,
          'y_pixel_size': 0.0045,
          'incident_wavelength': 1.28481,
          'sx': 256,
          'sy':256,
         'frame_time': tbins, 
          'count_time': tbins,
           'uid': 'uid=Run11'
         }

    filename = out_dir0 + uid + '_tbin=%s.cmp'%tbins
    if False:#consider dead time issue
        avg_img, imgsum, N = compress_timepix_data( poseff[good_time_ind],
                                               t0eff[good_time_ind],
                                               binstep, filename = filename, 
                                               md = md, force_compress= True  )
    else:        
        avg_img, imgsum, N = compress_timepix_data( poseff,
                                                    t0eff,
                                               binstep, filename = filename, 
                                               md = md, force_compress= True  )
        
    print(N)
    FD = Multifile(filename, 0, N  )
    uidstr_ = uidstr + 'tbins=%.3f_ms'%( tbins*1000 ) 
    
    plot1D( x = tbins* np.arange(len(imgsum)), 
           y= imgsum , m='o', c='b', ls = '--',save=True, path= out_dir,
         title='%s_imgsum_time'%(uidstr),
           xlabel = r"$time $ $(s)$", ylabel = 'Photon Counts of Image Sum' )
    
    # Check one ROI intensity
    roi_inten = check_ROI_intensity( avg_img, roi_mask, ring_number= 7, uid =uidstr_ ) #roi starting from 1
    # Do one time
    if use_sqnorm:norm = get_pixelist_interp_iq( qp_saxs, iq_saxs, roi_mask, center)
    else:norm=None 
    if use_imgsum_norm:imgsum_ = imgsum
    else:imgsum_ = None    
    import time
    if run_one_time: 
        t0 = time.time()    
        g2, lag_steps  = cal_g2p( FD,  roi_mask, bad_frame_list,good_start, num_buf = 8, num_lev= None,
                                imgsum= imgsum_, norm=norm )
        run_time(t0)

    lag_steps = lag_steps[:g2.shape[0]]
    uid_ = uidstr + '_fra_%s_%s_tbins=%.3f_ms'%(FD.beg, FD.end,  tbins*1000 )
    timeperframe  = tbins
    print(uid_)

    if run_one_time:

        taus = lag_steps * timeperframe    
        try:
            g2_pds = save_g2_general( g2, taus=taus,qr= np.array( list( qval_dict.values() ) )[:,0],
                                                qz = np.array( list( qval_dict.values() ) )[:,1],
                                 uid=uid_+'_g2.csv', path= out_dir, return_res=True )
        except:
            g2_pds = save_g2_general( g2, taus=taus,qr= np.array( list( qval_dict.values() ) )[:,0],                                             
                                 uid=uid_+'_g2.csv', path= out_dir, return_res=True )

    if run_one_time:
        g2_fit_result, taus_fit, g2_fit = get_g2_fit_general( g2,  taus, 
                    function = fit_g2_func,  vlim=[0.95, 1.05], fit_range= None,  
                fit_variables={'baseline':True, 'beta': True, 'alpha':False,'relaxation_rate':True},                                  
                guess_values={'baseline':1.0,'beta': 0.1,'alpha':1.0,'relaxation_rate':0.0100,},
                guess_limits = dict( baseline =[1, 1.8], alpha=[0, 2],
                            beta = [0, 1], relaxation_rate= [0.00001, 5000]) ) 
        g2_fit_paras = save_g2_fit_para_tocsv(g2_fit_result,  filename= uid_  +'_g2_fit_paras.csv', path= out_dir )

     

    if run_one_time:
        plot_g2_general( g2_dict={1:g2, 2:g2_fit}, taus_dict={1:taus, 2:taus_fit}, vlim=[0.95, 1.05],
                    qval_dict = qval_dict, fit_res= g2_fit_result,  geometry= scat_geometry_,filename= uid_+'_g2', 
            path= out_dir, function= fit_g2_func,  ylabel='g2', append_name=  '_fit')
  

    if run_one_time:
        if False:
            fs, fe = 0, 9
            fs,fe=0, 12
            qval_dict_ = {k:qval_dict[k] for k in list(qval_dict.keys())[fs:fe]  }
            D0, qrate_fit_res = get_q_rate_fit_general(  qval_dict_, g2_fit_paras['relaxation_rate'][fs:fe], 
                                                       geometry=  scat_geometry_ )
            plot_q_rate_fit_general( qval_dict_, g2_fit_paras['relaxation_rate'][fs:fe],  qrate_fit_res, 
                                    geometry= scat_geometry_,uid=uid_  , path= out_dir )
        else:
            D0, qrate_fit_res = get_q_rate_fit_general(  qval_dict, g2_fit_paras['relaxation_rate'],
                                        fit_range=[0, 26],   geometry= scat_geometry_ )    
            plot_q_rate_fit_general( qval_dict, g2_fit_paras['relaxation_rate'],  qrate_fit_res,   
                                geometry=  scat_geometry_,uid=uid_  ,
                                    show_fit= True, path= out_dir, plot_all_range=False)

    # Calculation of g2

    get_diffusion_coefficient(   8.9*1e-4, 125*1e-9, T=298)/1e8 

    num_bufs=8
    noframes =  1e6
    num_levels = int(np.log( noframes/(num_bufs-1))/np.log(2) +1) +1
    tot_channels, lag_steps2, dict_lag = multi_tau_lags(num_levels, num_bufs)
    max_taus= lag_steps2.max()
    taus_ = lag_steps2 * 1e-7
    g2_q1 = cal_particle_g2( radius=125*1e-9, viscosity=8.9*1e-4, qr=qr,  taus=taus_, beta=0.214, T=298)

    fig, ax = plt.subplots()
    for i, q1 in enumerate(qr):
        plot1D( x=taus_, y= g2_q1[i],   ls='-', m='', logx=True, ax =ax, c=colors[i],  legend=qr[i])
        plot1D( x=taus[1:], y= g2[1:,i], m = markers[i], ls='', logx=True, xlim=[0.000008, 0.01], ylim=[0.98,1.22],
               ax =ax, c=colors[i],   legend= '')
    fig.savefig(out_dir + uid_ + 'g2_with_cal.png')

    # Two time
    define_good_series = False
    #define_good_series = True
    if define_good_series:
        good_start = 5
        FD = Multifile(filename, beg = good_start, end = 1000)
        uid_ = uidstr + '_fra_%s_%s'%(FD.beg, FD.end)
        print( uid_ )

    data_pixel = None
    if run_two_time:    
        data_pixel =   Get_Pixel_Arrayc( FD, pixelist,  norm= norm ).get_data()

    import time
    t0=time.time()
    g12b=None
    if run_two_time:     
        g12b = auto_two_Arrayc(  data_pixel,  roi_mask, index = None   )
        if run_dose:
            np.save( out_dir + 'uid=%s_g12b'%uid, g12b)
    run_time( t0 )

    if run_two_time:
        show_C12(g12b, q_ind= 7, N1= FD.beg,logs=False, N2=min( FD.end,10000), vmin= 1.01, vmax=1.4, 
                 timeperframe=timeperframe,save=True, path= out_dir, uid = uid_ )

    multi_tau_steps = True
    if run_two_time:
        if lag_steps is None:
            num_bufs=8
            noframes = FD.end - FD.beg
            num_levels = int(np.log( noframes/(num_bufs-1))/np.log(2) +1) +1
            tot_channels, lag_steps, dict_lag = multi_tau_lags(num_levels, num_bufs)
            max_taus= lag_steps.max()

        #max_taus= lag_steps.max()  
        max_taus = N #Nimg    
        t0=time.time()
        #tausb = np.arange( g2b.shape[0])[:max_taus] *timeperframe
        if multi_tau_steps:
            lag_steps_ = lag_steps[   lag_steps <= g12b.shape[0] ]
            g2b = get_one_time_from_two_time(g12b)[lag_steps_]
            tausb = lag_steps_ *timeperframe
        else:
            tausb = (np.arange( g12b.shape[0]) *timeperframe)[:-200]
            g2b = (get_one_time_from_two_time(g12b))[:-200]
        run_time(t0)

        g2b_pds = save_g2_general( g2b, taus=tausb, qr= np.array( list( qval_dict.values() ) )[:,0],
                                  qz=None, uid=uid_ +'_g2b.csv', path= out_dir, return_res=True )

    if run_two_time:    
        g2b_fit_result, tausb_fit, g2b_fit = get_g2_fit_general( g2b,  tausb, 
                    function = fit_g2_func,  vlim=[0.95, 1.05], fit_range= None,  
                fit_variables={'baseline':False, 'beta': True, 'alpha':False,'relaxation_rate':True},                                  
                guess_values={'baseline':1.0,'beta': 0.15,'alpha':1.0,'relaxation_rate':1,},
                guess_limits = dict( baseline =[1, 1.8], alpha=[0, 2],
                            beta = [0, 1], relaxation_rate= [0.000001, 5000]) ) 
        g2b_fit_paras = save_g2_fit_para_tocsv(g2b_fit_result,  filename= uid_  +'_g2b_fit_paras.csv', path= out_dir )

    if run_two_time:
        plot_g2_general( g2_dict={1:g2b, 2:g2b_fit}, taus_dict={1:tausb, 2:tausb_fit}, vlim=[0.95, 1.05],
                    qval_dict=qval_dict, fit_res= g2b_fit_result,  geometry=scat_geometry_,filename=uid_+'_g2', 
                        path= out_dir, function= fit_g2_func,  ylabel='g2', append_name=  '_b_fit')

    if run_two_time:    
        if False:
            fs, fe = 0,9
            fs, fe = 0,12
            qval_dict_ = {k:qval_dict[k] for k in list(qval_dict.keys())[fs:fe]  }
            D0b, qrate_fit_resb = get_q_rate_fit_general(  qval_dict_, g2b_fit_paras['relaxation_rate'][fs:fe], geometry= scat_geometry_ )
            plot_q_rate_fit_general( qval_dict_, g2b_fit_paras['relaxation_rate'][fs:fe],  qrate_fit_resb, 
                                geometry= scat_geometry_,uid=uid_ +'_two_time' , path= out_dir )
        else:

            D0b, qrate_fit_resb = get_q_rate_fit_general(  qval_dict, g2b_fit_paras['relaxation_rate'],
                                            fit_range=[1, 10],  geometry= scat_geometry_ )
            plot_q_rate_fit_general( qval_dict, g2b_fit_paras['relaxation_rate'],  qrate_fit_resb,   
                                geometry= scat_geometry_,uid=uid_ +'_two_time', show_fit=False,path= out_dir, plot_all_range= True )

    if run_two_time and run_one_time:
        plot_g2_general( g2_dict={1:g2, 2:g2b}, taus_dict={1:taus, 2:tausb},vlim=[0.99, 1.007],
                    qval_dict=qval_dict, g2_labels=['from_one_time', 'from_two_time'],
                geometry=scat_geometry_,filename=uid_+'_g2_two_g2', path= out_dir, ylabel='g2', )



# Time Depedent I(q) Analysis

In [ ]:
if scat_geometry =='saxs':
    Nimg = FD.end - FD.beg 
    time_edge = create_time_slice( Nimg, slice_num= 8, slice_width= 2, edges = None )
    time_edge =  np.array( time_edge ) + good_start
    #print( time_edge )    
    qpt, iqst, qt = get_t_iqc( FD, time_edge, mask, pargs=setup_pargs, nx=1500, show_progress= False )
    plot_t_iqc( qt, iqst, time_edge, pargs=setup_pargs, xlim=[qt.min(), qt.max()],
           ylim = [iqst.min(), iqst.max()], save=True )

# Do a waterfall analysis

In [ ]:


ring_avg = None    
if run_t_ROI_Inten:
    times_roi, mean_int_sets = cal_each_ring_mean_intensityc(FD, roi_mask, timeperframe = None, multi_cor=True  ) 
    plot_each_ring_mean_intensityc( times_roi, mean_int_sets,  uid = uidstr, save=True, path= out_dir )
    roi_avg = np.average( mean_int_sets, axis=0)